# Amazon Bedrock AgentCore Memory

## Introduction

This notebook demonstrates how to add memory (short-term and long-term) to a multi-agent system built with Strands using AgentCore Memory. 

What you will learn:

* Create a shared memory resource that multiple agents can access
* Configure supervisor agent to store user visible conversations to the memory 
* Configure supervisor agent and sub-agents to retrieve recent conversation history from short-term memory as context
* Configure long-term memory strategy and manage data segregation with namespaces
* Congiture agent to retrieve user query related long term memory and use it as context


## Scenario context

In the previous session, you have taken the mortgage assistant built with Strands to Agentcore Runtime, added tools on Agentcore Gateway, authentication and authorisation with Agentcore Idenity. In this session, we continue the journey to make the agent more helpful by adding a memory component using Agentcore Memory.

## Architecture

![agentcore_memory_architecture.png](images/agentcore_memory_architecture.png)

1. Whenever a user visible conversation is added, whether it is from the user or agent, the conversation will be stored in AgentCore Memory as short-term memory. In Strands `Agents as Tools` setup, all visible conversation will pass through the supervisor agent.
2. Whenever an agent starts, it loads the last 3 conversation from short-term memory as context.
3. A memory extraction module works asynchronously at the background, and use the defined strategy to turn short-term memory into long-term memory.
4. Whenever a user asks a question, the supervisor agent will run a natural language query against the long-term memory and obtain related information as context. 

## Step 0: Environment Setup

In [ ]:
import logging
from datetime import datetime
import boto3
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("agentcore-memory")

## Step 1: Create Agentcore Memory Resource
First, create Agentcore Memory resource for the mortgage assistant in your AWS account.

In [ ]:
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

client = MemoryClient(region_name=region)
memory_name = "mortgage_assistant"
memory_id = None

In [ ]:
# for testing only: if agentcore memory has already been created outside this workshop
# memory_id="mortgage_assistant_20250817122625-BgMl7M34lQ"

In the following code, we create the Agentcore Memory resource.

Notice we create `strategies` here and use the them when creating Agentcore Memory. When `strategies` are provided, Agentcore Memory will use the provided strategies to process short-term memories (conversation histories) and generate long-term memories, such as user facts, user preference or conversation summaries.

If Agentcore Memory is created without `strategies`, it only keeps short-term memories (conversation histories). No long-term memories will be generated.

The Agentcore Memory resource will take a few minutes to create. 

In [ ]:
from botocore.exceptions import ClientError

try:
    print("Creating Memory...")
    memory_name = memory_name

    # Strategies for long term memory
    strategies = [
        {
            StrategyType.SEMANTIC.value: {
                "name": "fact_extractor",
                "description": "Extracts and stores factual information",
                "namespaces": ["mortgage_assistant/{actorId}/facts"],
            },
        },
        {
            StrategyType.SUMMARY.value: {
                "name": "conversation_summary",
                "description": "Captures summaries of conversations",
                "namespaces": ["mortgage_assistant/{actorId}/{sessionId}"],
            },
        },
        {
            StrategyType.USER_PREFERENCE.value: {
                "name": "user_preferences",
                "description": "Captures user preferences and settings",
                "namespaces": ["mortgage_assistant/{actorId}/preferences"],
            },
        },
    ]

    # Create the memory resource
    memory = client.create_memory_and_wait(
        name=memory_name,                       # Unique name for this memory store
        description="Mortgage Assistant Memory", # Human-readable description
        strategies=strategies,                  # Strategies is for long term memory. For short term memory only, pass in an empty array 
        event_expiry_days=7,                    # Memories expire after 7 days
        max_wait=300,                           # Maximum time to wait for memory creation (5 minutes)
        poll_interval=10                        # Check status every 10 seconds
    )

    # Extract and print the memory ID
    memory_id = memory['id']
    print(f"Memory created successfully with ID: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    print(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()

    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

Now the agentcore memory is created. You can go to your AWS Console -> Amazon Bedrock Agentcore -> Memory to see the resource. 

Please note, `memory_id` will be used in most API calls to store/retrieve memory. We will store the memory ID in System Manager Parameter Store, so our Agentcore Runtime can fetch this memory ID at runtime to access the Agentcore Memory we created.

In [ ]:
ssm = boto3.client("ssm")
param_name = "/app/mortgage_assistant/agentcore/memory_id"

def store_memory_id_in_ssm(param_name: str, memory_id: str):
    ssm.put_parameter(Name=param_name, Value=memory_id, Type="String", Overwrite=True)
    print(f"Stored memory_id in SSM: {param_name}")

In [ ]:
store_memory_id_in_ssm(param_name, memory_id)

## Step 2: Create Memory Hook Providers
Here we create classes for memory hooks. We tap into Strands agents native lifecycle events. In the relevant lifecycle events, we retrieve/store conversations. For more information about Strands agent's lifecycle events, please check [documentation on Strands Hooks](https://strandsagents.com/latest/documentation/docs/api-reference/hooks/).

The following code will be written into a python file `mortgage_agent_memory_hook.py`, which will be imported and used by the `mortgage_agent_runtime_with_memory.py` to create memory hook at runtime.

A few key points in the code.
1. Both `MortgageAssistantSupervisorMemoryHook` and `MortgageAssistantSubAgentMemoryHook` class override `on_message_added` method. 
2. In `on_message_added` overriden by `MortgageAssistantSupervisorMemoryHook`, we call `process_saving_message()` to save a message (if it is a user visiable message). We also call `process_long_term_memory_retrieval()` to retrieve long-term memories that is related to user queries.
3. In `on_message_added` overriden by `MortgageAssistantSubAgentMemoryHook`, we choose NOT to call `process_saving_message()` as the user messages are repeated, and are already saved by supervisor agent. Sub-agents can also generate internal messages, which supervisor agent may overwrite. So in this project, it is better to leave the responsibility of saving messages to the supervisor agent. Secondly, we don't need to retrieve long term memory again, because the supervisor agent has already done so and will pass that context to the sub-agents. In our implementation, we simply print out the message passed over from the supervisor agent in the log.

In [ ]:
%%writefile mortgage_agent_memory_hook.py
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent
from bedrock_agentcore.memory import MemoryClient
import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("agentcore-memory")


class MortgageAssistantMemoryHookAbstract(HookProvider):
    """An abstract class extending Strands provided HookProvider class. 

        This class should be extended and on_message_added() should be overriden to customise memory related behaviors
    """
    def __init__(self, agent_name: str, memory_client: MemoryClient, memory_id: str, actor_id: str, session_id: str):
        self.agent_name = agent_name
        self.memory_client = memory_client
        self.memory_id = memory_id
        self.actor_id = actor_id
        self.session_id = session_id

    def register_hooks(self, registry: HookRegistry) -> None:
        """Register memory hooks"""
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)
        
    def on_agent_initialized(self, event: AgentInitializedEvent):
        """Load recent conversation history when agent starts"""
        logger.info(f"[{self.__class__.__name__}] - {self.agent_name} - AgentInitializedEvent")
        logger.info(event)
        
        short_term_context = self.get_short_term_context()
        event.agent.system_prompt += short_term_context
        logger.info(f"[{self.__class__.__name__}] - {self.agent_name} - Short term context added to system prompt:")
        logger.info(f"[{self.__class__.__name__}] - {self.agent_name} - System_prompt: {event.agent.system_prompt}")

    def on_message_added(self, event: MessageAddedEvent):
        pass

    def process_saving_message(self, messages):
        # if the message does not have meaningful to store, skip 
        if "text" not in messages[-1]["content"][0].keys():
            logger.info(f"[{self.__class__.__name__}] - {self.agent_name} - Skipping creating memory record as this is no \"text\" in this message")
            return

        messages_to_store = [(messages[-1]["content"][0]["text"], messages[-1]["role"])]
        logger.info(f"[{self.__class__.__name__}] - {self.agent_name} - Creating a memory record")
        self.save_messages_to_memory(messages_to_store)

    def process_long_term_memory_retrieval(self, messages):
        # for a user query, find corresponding long term context, and add it to the message as context
        if messages[-1]["role"] == "user" and "toolResult" not in messages[-1]["content"][0]:
            logger.info(f"[{self.__class__.__name__}] - {self.agent_name} - fetching long term memory as context")
            user_query = messages[-1]["content"][0]["text"]
            long_term_context = self.get_long_term_context(user_query)
            messages[-1]["content"][0]["text"] = (
                f"Customer Context:\n{long_term_context}\n\n{user_query}"
            )
            logger.info(f"[{self.__class__.__name__}] - {self.agent_name} - long term context added to message:")
            logger.info(f"[{self.__class__.__name__}] - {self.agent_name} - {messages[-1]["content"][0]["text"]}")
        
    def save_messages_to_memory(self, messages_to_store):
        logger.info(f"[{self.__class__.__name__}] - {self.agent_name} - message to store: {messages_to_store}")
        logger.info(f"[{self.__class__.__name__}] - {self.agent_name} - Creating a memery record using create_event")
        logger.info(f"[{self.__class__.__name__}] - {self.agent_name} - memory_id: {self.memory_id}")
        logger.info(f"[{self.__class__.__name__}] - {self.agent_name} - actor_id: {self.actor_id}")
        logger.info(f"[{self.__class__.__name__}] - {self.agent_name} - session_id: {self.session_id}")
        
        try:
            self.memory_client.create_event(
                memory_id=self.memory_id,
                actor_id=self.actor_id,
                session_id=self.session_id,
                messages=messages_to_store
            )
            logger.info(f"[{self.__class__.__name__}] - {self.agent_name} - memory record created")

        except Exception as e:
            logger.error(f"[{self.__class__.__name__}] - {self.agent_name} - Failed to store message: {e}")
            logger.error(f"[{self.__class__.__name__}] - {self.agent_name} - Error occurred: {str(e)}")

    def get_long_term_context(self, user_query):
        long_term_context = ""
        try:
            # Retrieve customer facts and preference
            long_term_context_list = []
            
            namespaces_to_query = [f"mortgage_assistant/{self.actor_id}/facts", 
                                f"mortgage_assistant/{self.actor_id}/preferences"]
            for namespace in namespaces_to_query:
                logger.info(f"[{self.__class__.__name__}] - {self.agent_name} - get_long_term_context() | namespaces: {namespace}")
                
                memories = self.memory_client.retrieve_memories(
                    memory_id=self.memory_id,
                    namespace=namespace.format(actorId=self.actor_id),
                    query=user_query,
                    top_k=3
                )
                
                for memory in memories:
                    if isinstance(memory, dict):
                        content = memory.get('content', {})
                        if isinstance(content, dict):
                            text = content.get('text', '').strip()
                            if text:
                                long_term_context_list.append(text)
            
            # Inject customer context into the query
            if long_term_context_list:
                long_term_context = "\n".join(long_term_context_list)
                logger.info(f"Retrieved {len(long_term_context_list)} customer context records")
                
        except Exception as e:
            logger.error(f"Failed to retrieve long term memory: {e}")
            
        return long_term_context
        
    def get_short_term_context(self):
        short_term_context = ""
        try:
            # Get last 5 conversation turns
            recent_turns = self.memory_client.get_last_k_turns(
                memory_id=self.memory_id,
                actor_id=self.actor_id,
                session_id=self.session_id,
                k=5,
                branch_name="main"
            )

            if recent_turns:
                # Format conversation history for context
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        role = message['role'].lower()
                        content = message['content']['text']
                        context_messages.append(f"{role.title()}: {content}")

                context = "\n".join(context_messages)
                short_term_context = f"\n\nRecent conversation history:\n{context}\n\nContinue the conversation naturally based on this context."
            
                logger.info(f"[{self.__class__.__name__}] - {self.agent_name} - ✅ Loaded {len(recent_turns)} recent conversation turns")
                # logger.info(f"[{self.__class__.__name__}] - {self.agent_name} - short_term_context: {context}")
            else:
                logger.info(f"[{self.__class__.__name__}] - {self.agent_name} - No previous conversation history found")

        except Exception as e:
            logger.error(f"[{self.__class__.__name__}] - {self.agent_name} - Failed to load conversation history: {e}")
        
        return short_term_context
    

class MortgageAssistantSupervisorMemoryHook(MortgageAssistantMemoryHookAbstract):
    """Memory hook for supervisor agents in Strands, where all user visible messages will pass through.
    
        This class overriding on_message_added method to 
        1. save the message to agentcore memory
        2. retrieve long-term memory related to the user query
    """
    def on_message_added(self, event: MessageAddedEvent):
        logger.info(f"[{self.__class__.__name__}] - {self.agent_name} - MessageAddedEvent")
        messages = event.agent.messages
        self.process_saving_message(messages)
        self.process_long_term_memory_retrieval(messages)


class MortgageAssistantSubAgentMemoryHook(MortgageAssistantMemoryHookAbstract):
    """Memory hook for sub-agents in Strands, where user messages are repeated, and agent messages
        could be overwritten by the supervisor agent. 
    
        This class overriding on_message_added method to retrieve long-term memory related to the user query
    """
    def on_message_added(self, event: MessageAddedEvent):
        logger.info(f"[{self.__class__.__name__}] - {self.agent_name} - MessageAddedEvent")
        messages = event.agent.messages
        logger.info(f"[{self.__class__.__name__}] - {self.agent_name} - messages: {messages}")


## Step 3: Create Multi-Agent Architecture with Strands Agents
Now we can re-create the Agentcore Runtime for the Strands agents as you have done in the last few labs. The execution role for Agentcore Runtime needs access to the Agentcore Memory, and the System Manager Parameter Store to access the Memory ID. This is handled for you in this lab through a project-wide util function.

In [ ]:
from utils import create_agentcore_role

agent_name="mortgage_assistant"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

Now we can configure the Agentcore Runtime. 

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

agentcore_runtime = Runtime()
agent_name = "mortage_assistant_with_memory"
response = agentcore_runtime.configure(
    entrypoint="mortgage_agent_runtime_with_memory.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

Now we will launch the AgentCore Runtime resource in your AWS account. It will take a few minutes as the docker image is built in Code Build and deployed to Agentcore Runtime.

In [ ]:
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)

Now you can see the Agentcore Runtime resource in your AWS account in Amazon Bedrock Agentcore -> Agent Runtime. But it won't be ready until the status is changed to `READY`

As usual, we check the status of agentcore runtime until it is ready.

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
print(status)

Once the status is shown as `READY`, the Agentcore Runtime is ready to be invoked.

## Step 4: Invoke the deployed agent

First, let's create a `user_id` and `session_id` for testing. We also create `user_id_for_testing_no_memory` and `session_id_for_testing_no_memory` to test memory isolation. 

In [ ]:
import uuid

# An randomly generated fictional user ID. 
# In a real system, replace this with a real user ID in your system
user_id=str(uuid.uuid4())
user_id_for_testing_no_memory=str(uuid.uuid4())

session_id = str(uuid.uuid4())
session_id_for_testing_no_memory = str(uuid.uuid4())

print(f"user_id: {user_id}")
print(f"user_id_for_testing_no_memory: {user_id_for_testing_no_memory}")
print(f"session_id: {session_id}")
print(f"session_id_for_testing_no_memory: {session_id_for_testing_no_memory}")

We create a convenient method for printing the reponse messages nicely.

In [ ]:
def print_response_text(invoke_response):
    response_bytes=invoke_response['response']
    response_strings = [item.decode('utf-8') for item in response_bytes]
    # print(response_bytes)
    response_combined_string = "".join(response_strings)
    # print(response_combined_string)
    
    response_dict = json.loads(response_combined_string)
    response_text=response_dict["result"]["content"][0]["text"]
    print(response_text)

In [ ]:
sample_prompt_1 = "Hey I am John. I'd like to check my existing home loan. My customer id is ABC-123"
sample_prompt_2 = "We are planning to upsize. We are shopping around to see what is avaialble?"
sample_prompt_3 = "Do do banks decide how much it can lend me?"
sample_prompt_4 = "What is the difference between standard variable and fixed rate"
sample_prompt_5 = "What is interest only vs principal and interest?"

Now we call the agent a few times. 

In [ ]:
import pprint
import json

invoke_response = agentcore_runtime.invoke({
    "prompt": sample_prompt_1,
    "user_id": user_id
}, session_id=session_id)
# pprint.pprint(invoke_response, width=80, depth=None)
print_response_text(invoke_response)

In [ ]:
invoke_response = agentcore_runtime.invoke({
    "prompt": sample_prompt_2,
    "user_id": user_id
}, session_id=session_id)
print_response_text(invoke_response)

In [ ]:
invoke_response = agentcore_runtime.invoke({
    "prompt": sample_prompt_3,
    "user_id": user_id
}, session_id=session_id)
print_response_text(invoke_response)

In [ ]:
invoke_response = agentcore_runtime.invoke({
    "prompt": sample_prompt_4,
    "user_id": user_id
}, session_id=session_id)
print_response_text(invoke_response)

## Step 4: Check memory

After calling the agents a few times, we can now check the memory. We use two ways to check the memory.

1. We will check the memory with the user_id and session_id with Bedrock API. This allows us to check the stored short-term and long-term memory.
2. We will check the CloudWatch log to see the exact memory operations at the lifecycle events in supervisor and sub-agents. This helps us to adapt our memory management to suit our use case.

### 4.1 Check short-term memory
First, we create a convenient method for listing out the last K conversation turns from the short-term memory.

In [ ]:
# Check what's stored in memory
def list_last_k_turns(memory_id, actor_id, session_id, k):
    print("=== Memory Contents ===")
    print(f"actor_id: {actor_id}")
    print(f"session_id: {session_id}")
    
    recent_turns = client.get_last_k_turns(
        memory_id=memory_id,
        actor_id=actor_id,
        session_id=session_id,
        k=k # Adjust k to see more or fewer turns
        # branch_name="main"
    )
    
    for i, turn in enumerate(recent_turns, 1):
        print(f"Turn {i}:")
        for message in turn:
            role = message['role']
            content = message['content']['text'][:200] + " <🖨️ ... omitted for printing only ... 🖨️>" if len(message['content']['text']) > 200 else message['content']['text']
            print(f"  {role}: {content}")
        print()

Using a valid `user_id` and `session_id`, we expect to see the recent conversation history.

In [ ]:
list_last_k_turns(memory_id, f"{user_id}", session_id, 10)

Next, check the memory with a different user_id `user_id_for_testing_no_memory` which does not exist. We expect to see nothing.

In [ ]:
list_last_k_turns(memory_id, f"{user_id_for_testing_no_memory}-supervisor", session_id, 10)

Check the memory with the same `user_id`, but a different session_id `session_id_for_testing_no_memory`. Again, we expect to see NO messages.

In [ ]:
list_last_k_turns(memory_id, f"{user_id}", session_id_for_testing_no_memory, 10)

#### CloudWatch Log
Let's find the the log file in CloudWatch. You should see the short-term memory loaded at `AgentInitializedEvent`. 

![log-short-term-memory-loaded.png](images/log-short-term-memory-loaded.png)

You should also see at `MessageAddedEvent`, the current conversation is stored to memory.

![log-store-memory.png](images/log-store-memory.png)

### 4.2 Check long-term memory
Again, we will first use `list_memory_records` method to see the long-term memory.

In [ ]:
agentcore_client = boto3.client("bedrock-agentcore")

print("----------")
print("Long term memory - facts:")
response_facts = agentcore_client.list_memory_records(
    memoryId=memory_id,
    namespace=f"mortgage_assistant/{user_id}/facts",
    maxResults=10
)
# pprint.pprint(response_facts, width=80, depth=None)
for list_memory_record in response_facts["memoryRecordSummaries"]:
    print(f"Content: {list_memory_record['content']['text']}")
print("----------")

print("Long term memory - preferences:")
response_preferences = agentcore_client.list_memory_records(
    memoryId=memory_id,
    namespace=f"mortgage_assistant/{user_id}/preferences",
    maxResults=10
)
# pprint.pprint(response_preferences, width=80, depth=None)
for list_memory_record in response_preferences["memoryRecordSummaries"]:
    print(f"Content: {list_memory_record['content']['text']}")
print("----------")

print("Long term memory - summary:")
response_summary = agentcore_client.list_memory_records(
    memoryId=memory_id,
    namespace=f"mortgage_assistant/{user_id}/{session_id}",
    maxResults=10
)
# pprint.pprint(response_summary, width=80, depth=None)
for list_memory_record in response_summary["memoryRecordSummaries"]:
    print(f"Content: {list_memory_record['content']['text']}")
print("----------")

Alternatively, we can also use `retrieve_memories` method to retrieve long-term memory.

In [ ]:
memory_client=MemoryClient(region_name=region)
print("----------")
print("Long term memory - facts:")
facts_memory=memory_client.retrieve_memories(
            memory_id=memory_id, namespace=f"mortgage_assistant/{user_id}/facts", query="all", top_k=10
        )
# to see the raw facts memory records
# print(facts_memory)
# print the content
for memory in  facts_memory:
    print(memory["content"]["text"])

print("----------")
print("Long term memory - preferences:")
prefs_memory=memory_client.retrieve_memories(
            memory_id=memory_id, namespace=f"mortgage_assistant/{user_id}/preferences", query="all", top_k=10
        )
# print the content
for memory in  prefs_memory:
    memory_content = memory["content"]["text"]
    memory_content_json = json.loads(memory_content)
    print(json.dumps(memory_content_json, indent=4))

print("----------")
print("Long term memory - summaries:")
summaries_memory=memory_client.retrieve_memories(
            memory_id=memory_id, namespace=f"mortgage_assistant/{user_id}/{session_id}", query="all", top_k=10
        )
# print the content
for memory in  summaries_memory:
    print(memory["content"]["text"])
print("----------")

#### CloudWatch
Let's find the the log file in CloudWatch. You should see the short-term memory loaded at `MessageAddedEvent`.

![log-long-term-memory-loaded.png](images/log-long-term-memory-loaded.png)

#### 🎉 Congratulation! 
You have added memory to your multi-agent Mortgage Assistant. 

Explore more on the Strands agents lifecycle events, experiment with different ways to use Agentcore Memory, and make your AI agent more intelligent!

You can now move to the next session.